In [56]:
import pandas as pd
import requests
from collections import defaultdict
from pathlib import Path

In [57]:
year = 2022
leaderboard_id = 976545
token_path = Path.home() / '.config/aocd/token'

In [58]:
with open(token_path) as f:
    token = f.read().strip()

cookies = {'session': token}
headers = {'User-Agent': 'github.com/attoPascal'}
response = requests.get(f'https://adventofcode.com/{year}/leaderboard/private/view/{leaderboard_id}.json', cookies=cookies, headers=headers).json()

In [59]:
time_diffs = defaultdict(lambda: defaultdict(str))

for id, member in response['members'].items():
    name = member['name'] if member['name'] else id
    for day, times in member['completion_day_level'].items():
        if '1' in times and '2' in times:
            time_diffs[name][int(day)] = int(times['2']['get_star_ts']) - int(times['1']['get_star_ts'])

In [60]:
df = pd.DataFrame.from_dict(time_diffs, orient='index', dtype='Int64')
df.sort_index(key=lambda x: x.str.lower(), inplace=True)
df.sort_index(axis=1, inplace=True)

In [61]:
df

,1,2,3,4,5
1139456,18,258,160,163,11
1614144,68,7,32,61,577
2316524,3353,<NA>,<NA>,<NA>,<NA>
740740,1001,976,7452,325,<NA>
999754,434,1572,680,750,<NA>
Alex Ji,914,530,<NA>,<NA>,<NA>
attoPascal,117,513,444,267,394
Azratosh,120,103,247,181,456
benjo121ben,419,952,222,1083,<NA>
Benni S,475,788,2688,276,628


In [62]:
num_members, num_days = df.shape
points = pd.Series(0, index=df.index, dtype=int)

for day in range(num_days):
    points_awarded = num_members
    for name, time in df[day+1].sort_values().iteritems():
        if not pd.isna(time):
            points[name] += points_awarded
            points_awarded -= 1

In [63]:
# part 2 leaderboard
points.sort_values(ascending=False).head(20).to_frame()

,0
1139456,279
1614144,270
Azratosh,248
felpower,238
msonnberger,238
RaphaelTarita,234
TechnikTobi,230
devgertschi,225
David Mikan,224
cluosh,221


In [64]:
# seconds for each part 2
df.loc['attoPascal'].to_frame()

,attoPascal
1,117
2,513
3,444
4,267
5,394


In [65]:
# seconds for latest part 2
df.iloc[:, -1].dropna().sort_values().to_frame()

,5
1139456,11
felpower,51
Der_Himbeerkuchen,67
NotsoblackReaper,79
cluosh,84
JohnBryte,98
David Wild,120
devgertschi,135
thoosequa,150
tomato-alex,190


In [66]:
# sum for all parts 2 in hours
df.sum(axis=1, skipna=False).dropna().sort_values().to_frame() / 3600

,0
1139456,0.169444
1614144,0.206944
Azratosh,0.3075
msonnberger,0.370556
felpower,0.381944
TechnikTobi,0.417778
RaphaelTarita,0.434167
devgertschi,0.45
NotsoblackReaper,0.473889
attoPascal,0.481944
